# Naver financial news scraping + analyzing

In this notebook, you will use bs4 to get news titles from Naver financial news.

In [71]:
import pandas as pd
from datetime import datetime

import requests
import urllib
from bs4 import BeautifulSoup

from tqdm import tqdm

In [72]:
datelist = pd.date_range(start="2022-05-01", end="2022-05-10").tolist()

In [73]:
datelist = [int(date.strftime("%Y%m%d")) for date in datelist]

In [74]:
datelist # date 형태 int

[20220501,
 20220502,
 20220503,
 20220504,
 20220505,
 20220506,
 20220507,
 20220508,
 20220509,
 20220510]

In [101]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'# type your own user-agent here
}

In [102]:
def get_soup(date, page, headers):
  """
  Input
  - date(int): date in integer type
  - page(int): number of page
  - headers(dict): dictionary of headers for scraping

  Output
  - soup(bs4.Soup)
  """
  url = f"https://news.naver.com/main/list.naver?mode=LS2D&sid2=262&sid1=101&mid=shm&date={date}&page={page}"

  response = requests.get(url, headers=headers)

  soup = BeautifulSoup(response.text, 'html.parser')

  return soup

In [103]:
def get_titles_list(x):
  
  """
  function to get list of news titles in the current url

  Input
  - x: soup
  Output
  - titles: list of news titles in current url
  """
  
  headline = x.find_all('dl')

  titles = []
  for item in headline:
    try:
      title = item.find('dt', {'class' : 'photo'}).find('img')['alt']
      titles.append(title)
    except:
      None
  
  return titles

In [104]:
titles_dict = {}

for date in tqdm(datelist, desc="Getting requests"):
  
  page = 1

  soup = get_soup(date, page, headers)

  # number of pages for current date
  num_iterations = len(soup.find_all('a', {'class': 'nclicks(fls.page)'}))

  titles = get_titles_list(soup)

  for _ in range(num_iterations):

    page += 1
    
    soup = get_soup(date, page, headers)

    cur_titles = get_titles_list(soup)

    titles = titles + cur_titles
  
  titles_dict[date] = titles

Getting requests: 100%|██████████| 10/10 [00:22<00:00,  2.28s/it]


In [106]:
titles_dict.keys()

dict_keys([20220501, 20220502, 20220503, 20220504, 20220505, 20220506, 20220507, 20220508, 20220509, 20220510])

In [107]:
len(titles_dict.values())

10

In [108]:
import itertools

merged_titles = list(itertools.chain.from_iterable([*titles_dict.values()]))

In [127]:
len(merged_titles)

870

In [128]:
merged_titles[-1]

'[사진] 중국 잇단 도시 봉쇄 후폭풍, 4월 수출증가율 2년래 최저'

## Preprocessing

In [129]:
import nltk

In [130]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [131]:
from nltk.corpus import stopwords

In [132]:
!pip install konlpy

In [133]:
from konlpy.tag import Okt

In [134]:
okt = Okt()

In [137]:
okt.morphs(merged_titles[-1])

['[',
 '사진',
 ']',
 '중국',
 '잇단',
 '도시',
 '봉쇄',
 '후',
 '폭풍',
 ',',
 '4월',
 '수출',
 '증가',
 '율',
 '2년',
 '래',
 '최저']

In [138]:
# let's remove numbers and special characters
import re

ch_removed_titles = [re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣ]", " ", title) for title in merged_titles]

In [139]:
tokenized_titles = [okt.morphs(title) for title in ch_removed_titles]

In [140]:
tokenized_titles[-1]

['사진', '중국', '잇단', '도시', '봉쇄', '후', '폭풍', '월', '수출', '증가', '율', '년', '래', '최저']

In [141]:
# Source: https://mr-doosun.tistory.com/24
stopwords = '아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 은 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓'.split(' ')

In [142]:
titles_final = [[word for word in title if word not in stopwords] for title in tokenized_titles]

In [143]:
titles_final[-1]

['사진', '중국', '잇단', '도시', '봉쇄', '폭풍', '수출', '증가', '율', '래', '최저']

## Keyword Analysis

There are a lot of ways that you can extract keywords from a document. For those of you who are interested in this topic, I highly recommend you to google **PageRank** and **TextRank** algorithm. Similar concept such as **Collaborative Filtering** is used in recommender system. 

Instead of using fancy algorithms, in this notebook, we will use näive Counter model. Idea is pretty simple. If two words A and B co-occur a lot, it's more likely to say that they are highly correlated and B is a related keyword for A.

In [144]:
titles_final[:10]

[['홍콩', '코로나', '신규', '감염', '사흘', '연속', '명대', '총'],
 ['올', '댓', '차이나', '신축', '주택', '가격', '지방', '정책', '완화'],
 ['상하이', '조업', '재개', '차', '대상', '기업', '통보', '정상화', '속도'],
 ['올', '댓', '차이나', '올해', '분기', '중국', '철강', '소', '비량', '코로나', '재', '확산', '여파'],
 ['중국', '국제사회', '제재', '해외', '자산', '보호', '대책', '긴급', '협의'],
 ['글로벌', '증시', '하락', '장', '계속', '워런', '버핏', '분기', '조원', '손실'],
 ['베트남', '양조', '업체', '베코', '분기', '순이익', '조', '천', '억동', '년비', '증가'],
 ['기시다', '총리', '베트남', '공식', '방문', '하노이', '도착'],
 ['베트남', '포스코', '호', '팟', '그룹', '분기', '순이익', '조', '천', '억동', '년비', '증가'],
 ['베트남', '분기', '디지털', '경제', '수익', '억', '달러', '추산']]

In [145]:
us = [title for title in titles_final if '미국' in title]

In [146]:
us

[['팬데믹', '기간', '있는', '돈', '불었다', '미국', '주택', '자산', '조', '급등'],
 ['버핏', '전쟁', '인플레', '압박', '에도', '미국', '매입', '래', '최대'],
 ['미국', '연말', '물가상승률', '돼야', '경기', '침체', '피'],
 ['미국', '전기차', '배터리', '부문', '인프라', '자금', '조원', '투입'],
 ['미국', '년물', '국채', '수익률', '돌파', '의미', '는'],
 ['올', '댓', '차이나', '대만', '증시', '미국', '기술', '강세', '반등', '마감'],
 ['스타벅스', '분기', '선방', '미국', '강한', '성장', '중국', '급감'],
 ['미국', '기업', '임금', '자극', '해', '인플레', '지속', '우려'],
 ['파월', '두어', '차례', '인상', '인플레', '와의', '전쟁', '시작', '미국'],
 ['홍콩도', '미국', '기준금리', '포인트', '인상'],
 ['미국', '연', '준', '만에', '기준금리', '인상', '긴축', '고삐'],
 ['속보', '미국', '연', '준', '만에', '기준금리', '인상', '긴축', '고삐'],
 ['미국', '업체', '일자리', '순', '증', '실업률', '같아', '종합'],
 ['미국', '증시', '급락', '하루', '새', '러시아', '증발'],
 ['미국', '지수선물', '소', '폭', '일제', '상승'],
 ['미국', '다우지수', '연속', '하락', '처음'],
 ['미국', '금리', '인상', '공포', '전세계', '증시', '덮', '쳐', '일제', '급락', '종합'],
 ['미국',
  '금리',
  '중국',
  '코로나',
  '봉쇄',
  '재등장',
  '스',
  '태그',
  '플',
  '래',
  '션',
  '우려',
  '신인',
  '규',
  '뉴욕증시',
  '나우'],

In [147]:
us_excluded = [word for word in list(itertools.chain.from_iterable([*us])) if word != '미국']

In [148]:
len(us_excluded)

196

In [149]:
us_excluded[:5]

['팬데믹', '기간', '있는', '돈', '불었다']

In [150]:
# let's count
import collections

num_keyword = 7
print(collections.Counter(us_excluded).most_common(num_keyword))

[('인상', 7), ('인플레', 5), ('일제', 4), ('금리', 4), ('증시', 3), ('중국', 3), ('기준금리', 3)]


### Visualize

In [41]:
!apt-get update -qq
!apt-get install fonts-nanum* -qq

Selecting previously unselected package fonts-nanum.
(Reading database ... 155203 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [42]:
!pip install wordcloud

In [43]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

In [44]:
import matplotlib.font_manager as fm

path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'  # install font
font_name = fm.FontProperties(fname = path, size=10).get_name()
print(font_name)
plt.rc('font',family = font_name)
fm._rebuild()

NanumGothic


In [45]:
# generate wordcloud
wc_title = WordCloud(font_path=path,width=5000, height=5000, scale=6.0, max_font_size=2500)
gen_title = wc_title.generate_from_frequencies(collections.Counter(us_excluded))
plt.figure()
plt.imshow(gen_title)

KeyboardInterrupt: ignored